In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

In [3]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

In [4]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [5]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [6]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

# Calculo los distintos algoritmos con los parametros que mejor resultado nos dio

In [7]:
### Bagging regressor

bag = BaggingRegressor(n_estimators = 5, max_samples = 1.0, max_features = 1.0)
bag.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado_Bagging_Regressor'] = set_pruebas.loc[:,columnas].apply(lambda x: bag.predict(x)[0],axis = 1)

In [8]:
### Ada boost

ada = AdaBoostRegressor(n_estimators = 3, loss = 'square')
ada.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado_Ada_Boost'] = set_pruebas.loc[:,columnas].apply(lambda x: ada.predict(x)[0],axis = 1)

In [9]:
### Decision Tree

tree = DecisionTreeRegressor(criterion = 'mse', max_depth = None, min_samples_split = 2,\
                             min_samples_leaf = 1, max_features = 1.0)
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado_Decision_Tree'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)

In [10]:
### KNN

knn = KNeighborsRegressor(n_neighbors = 1, p = 1)
knn.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado_KNN'] = set_pruebas.loc[:,columnas].apply(lambda x: knn.predict(x)[0],axis = 1)

In [11]:
### Perceptron

perceptron = MLPRegressor(solver = 'lbfgs', activation = 'logistic', tol = 1e-8)
perceptron.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado_Perceptron'] = set_pruebas.loc[:,columnas].apply(lambda x: perceptron.predict(x)[0],axis = 1)

In [12]:
### Random Forest

randomforest = RandomForestRegressor(n_estimators = 20, max_features = 0.4)
randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado_Random_Forest'] = set_pruebas.loc[:,columnas].\
                        apply(lambda x: randomforest.predict(x)[0],axis = 1)

# Calculo el promedio de los resultados con todas las combinaciones posibles entre algoritmos

In [13]:
algoritmos = ['KNN', 'Random_Forest', 'Decision_Tree', 'Bagging_Regressor','Perceptron','Ada_Boost']

In [14]:
# Prueba todas las combinaciones de algoritmos y calcula el promedio de ellos
# El set de pruebas debe tener una columna por algoritmo con su resultado
# Las columnas deben llamarse resutado_algoritmo

def calcular_promedio(lista, set_pruebas):
    set_pruebas.loc[:,'promedio'] = 0
    for i in range(len(lista)):
        columna = 'resultado_' + lista[i]
        set_pruebas.loc[:,'promedio'] = set_pruebas.loc[:,'promedio'] + set_pruebas.loc[:,columna]
    set_pruebas.loc[:,'promedio'] = set_pruebas.loc[:,'promedio'] / len(lista)
    error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.promedio)
    return error

def combinaciones(lista, minimo):
    comb = [[]]
    for x in lista:
        comb = comb + [c + [x] for c in comb]
    res = [c for c in comb if len(c) >= minimo]
    return res

In [15]:
errores = []
combinaciones_posibles = combinaciones(algoritmos,2)
for comb in combinaciones_posibles:
    error = calcular_promedio(comb,set_pruebas)
    errores.append((error,comb))
    print("Error = {}, algoritmos = {}".format(error,comb))

Error = 14266035004.118303, algoritmos = ['KNN', 'Random_Forest']
Error = 13447551683.010105, algoritmos = ['KNN', 'Decision_Tree']
Error = 13167017744.002684, algoritmos = ['Random_Forest', 'Decision_Tree']
Error = 13143630556.971027, algoritmos = ['KNN', 'Random_Forest', 'Decision_Tree']
Error = 15232589483.03835, algoritmos = ['KNN', 'Bagging_Regressor']
Error = 16525748799.911219, algoritmos = ['Random_Forest', 'Bagging_Regressor']
Error = 14613000295.9816, algoritmos = ['KNN', 'Random_Forest', 'Bagging_Regressor']
Error = 14108598239.029737, algoritmos = ['Decision_Tree', 'Bagging_Regressor']
Error = 13562536727.210936, algoritmos = ['KNN', 'Decision_Tree', 'Bagging_Regressor']
Error = 14116865245.021082, algoritmos = ['Random_Forest', 'Decision_Tree', 'Bagging_Regressor']
Error = 13559550563.35171, algoritmos = ['KNN', 'Random_Forest', 'Decision_Tree', 'Bagging_Regressor']
Error = 44330463621.53088, algoritmos = ['KNN', 'Perceptron']
Error = 48020057048.287575, algoritmos = ['Ran

In [16]:
min_error = float("inf")
min_e = ()
for e in errores:
    if e[0] < min_error:
        min_error = e[0]
        min_e = e[1]

print("Menor Error = {}, algoritmos = {}".format(min_error,min_e))

Menor Error = 13143630556.971027, algoritmos = ['KNN', 'Random_Forest', 'Decision_Tree']


# Tomamos los algoritmos con menor error

In [17]:
algoritmos = ['KNN','Random_Forest','Decision_Tree']
funciones = [knn,randomforest,tree]

In [18]:
error = calcular_promedio(algoritmos,set_pruebas)
print(error)

13143630557.0


# Calculo con los verdaderos datos a analizar

In [19]:
analizar = pd.read_csv('/home/mati/Desktop/properati_dataset_modificado.csv')

In [20]:
for i in range(len(algoritmos)):
    algoritmo = algoritmos[i]
    funcion = funciones[i]
    cadena = 'resultado_' + algoritmo 
    analizar.loc[:,cadena] = analizar.loc[:,columnas].apply(lambda x: funcion.predict(x)[0],axis = 1)

In [22]:
analizar.loc[:,'price_aprox_usd'] = 0
calcular_promedio(algoritmos, analizar)
analizar.rename(columns = {'Promedio': 'price_usd'}, inplace = True)

In [23]:
resultado = analizar.loc[:,['id','price_usd']]

In [24]:
resultado.to_csv('Ensamble_con_Promedios.csv', index = False)